In [ ]:
import numpy as np
import h5py 
import pandas as pd

import cv2
import matplotlib.pyplot as plt

def get_positions(fn):
    path = 'runs/'
    with h5py.File(path + '/'+fn+'.hdf5', 'r') as f:
        # read the data
        positions = f['x'][:]
        properties = f['properties'][:][0]
        p = f['p'][:]

    return positions



stas = pd.read_csv('Data/tracked_data_rescaled.csv')


# rotate the stas data 45 degrees around y
rot  = 0.055
stas['x_rot'] = stas.x*np.cos(rot) - stas.z*np.sin(rot)
stas['z_rot'] = stas.x*np.sin(rot) + stas.z*np.cos(rot)
stas['y_rot'] = stas.y




# find the stas movement vectors
stas_dirs = -np.ones((len(np.unique(stas.frame)),11000,3))*9999
stas_poss = -np.ones((len(np.unique(stas.frame)),11000,3))*9999
for i, frame in enumerate(np.unique(stas.frame)):
    dirs = np.array([stas[stas.frame == frame].dx, stas[stas.frame == frame].dy, stas[stas.frame == frame].dz]).T

    lens = np.linalg.norm(dirs, axis = 1)
    lens[lens == 0] = 1
    dirs = dirs/lens[:,None]
    
    stas_dirs[i][:len(dirs)] = dirs

    poss = np.array([stas[stas.frame == frame].x_rot, stas[stas.frame == frame].y_rot, stas[stas.frame == frame].z_rot]).T

    stas_poss[i][:len(poss)] = poss


def get_sim_dirs(positions):
        
    sim_step_size = 200

    sim_dirs = np.empty((positions.shape[0]-sim_step_size, positions.shape[1], 3))
    sim_poss = np.empty((positions.shape[0]-sim_step_size, positions.shape[1], 3))  
    for t in range(positions.shape[0]-sim_step_size):
        dir = positions[t+sim_step_size] - positions[t]
        dir = dir/np.linalg.norm(dir, axis = 1)[:,None]
        sim_dirs[t] = dir
        sim_poss[t] = positions[t]

    return sim_poss, sim_dirs


In [ ]:
def get_overlappings(fn):
    all_positions = get_positions(fn)

    sim_poss, sim_dirs = get_sim_dirs(all_positions)

    scale = int(sim_poss.shape[0]/100)

    iiis = list(range(0, 40))
    overlaps = np.empty((len(iiis), len(sim_poss[0])))
    avg_compared_dirs = np.empty((len(iiis), len(sim_poss[0]), 3))
    for ind, iii in enumerate(iiis):

        sim_t = iii*scale
        stas_t = iii +1

        overlap = np.empty((len(sim_poss[sim_t])))
        for cell_id, (pos, dir) in enumerate(zip(sim_poss[sim_t], sim_dirs[sim_t])):
            found = False
            dist = 3
            print(f"{ind/len(iiis):.3}  -  {cell_id/len(sim_poss[sim_t]):.3}", end = "\r")
            while True:
                if dist > 50:
                    print(dist, pos)

                
                
                # find the all stas particles within a certain distance
                dists = np.linalg.norm(stas_poss[stas_t] - pos, axis = 1)

                closest = np.where(dists < dist)[0]


                dirs = stas_dirs[stas_t][closest]

                # remove vector if it contains nans
                dirs = dirs[~np.isnan(dirs).any(axis=1)]
                if len(dirs) < 2:
                    dist += 0.1
                    continue
                
                avg_dir = np.mean(dirs, axis = 0)
                avg_compared_dirs[ind][cell_id] = avg_dir

                dot = np.dot(dir, avg_dir)

                break

                    

            # find the angle between the sim dir and the stas dir
            angles = np.arccos(dot)

            overlap[cell_id] = np.mean(angles)
        overlaps[ind] = overlap

    return overlaps